In [ ]:
!pip uninstall tensorflow-model-optimization

In [ ]:
!pip install tensorflow-model-optimization

In [ ]:
import numpy as np
import os
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
!pip install --upgrade tensorflow-model-optimization
import tensorflow_model_optimization as tfmot

## Converting the already trained saved .pb model to tensorflow lite (tf-lite) model

In [ ]:
import os
import tensorflow as tf

# Path to the directory containing the SavedModel
saved_model_dir = "../model_versions/1"

# Check if the directory exists
if not os.path.exists(saved_model_dir):
    print(f"Error: Directory '{saved_model_dir}' does not exist.")
else:
    # Convert the SavedModel to TensorFlow Lite format
    try:
        converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        tflite_model = converter.convert()

        # Save the TensorFlow Lite model
        output_dir = 'tf_lite_models'
        os.makedirs(output_dir, exist_ok=True)
        tflite_model_filename = 'converted_model.tflite'
        tflite_model_path = os.path.join(output_dir, tflite_model_filename)
        with open(tflite_model_path, 'wb') as f:
            f.write(tflite_model)

        print("Model converted successfully and saved to:", tflite_model_path)
    except Exception as e:
        print("An error occurred during conversion:", str(e))


## ----using Quantized Aware Training

In [2]:
INIT_LR = 1e-3
BATCH_SIZE = 32
EPOCHS = 50
IMAGE_SIZE = 256
default_image_size = tuple((IMAGE_SIZE, IMAGE_SIZE))
image_size = 0
data_dir = "../../dataset_images""
CHANNELS=3
AUTOTUNE = tf.data.AUTOTUNE


In [3]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    ds_size = ds.cardinality().numpy()
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds


In [4]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=default_image_size,
  batch_size=BATCH_SIZE
)


train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [5]:
class_names = dataset.class_names
n_classes = len(class_names)
print(n_classes, class_names)

In [6]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

<CENTER><sub>
<h4>Using train_test_split from scikit-learn:</h4>
Especially when working with tabular data or using scikit-learn for machine learning tasks, there are built-in methods for splitting datasets into training, validation, and testing sets. The train_test_split function from scikit-learn is commonly used for this purpose.

```python
from sklearn.model_selection import train_test_split

# Assuming 'X' is your feature data and 'y' is your target labels
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=42)


<CENTER><sub>
<h4>Choosing Between the Methods:</h4>
If we're working with structured/tabular data and traditional machine learning algorithms, scikit-learn's train_test_split method is often more convenient and suitable. This method is well-established and efficient for splitting structured data into training and testing sets.

However, for complex data types such as images or for deep learning tasks, using a custom splitting function may be the better approach. Custom functions allow for more flexibility and control over how the dataset is divided, which is crucial for tasks like image classification where specific data preprocessing and augmentation may be needed.
</sub></CENTER>


<CENTER><sub>
<h4>Developing a Custom Dataset Distribution Function:</h4>
Since the task is image classification, we are developing our own function to distribute training, testing, and validation datasets. This is because scikit-learn is best suited for structured/tabular data and may not handle complex data like images efficientlyset, testing_dataset


In [7]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break


In [8]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

In [9]:
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255),
])

<CENTER><sub>
<h4>Caching the dataset:</h4>
Caching the dataset (training_data.cache()) helps speed up data loading by storing elements in memory or on disk, reducing the overhead of preprocessing during training epochs.

<h4>Shuffling the dataset:</h4>
Shuffling the dataset (training_data.shuffle(1000)) introduces randomness into the data, preventing the model from memorizing the order of samples and improving its generalization ability.

<h4>Prefetching data batches:</h4>
Prefetching data batches (training_data.prefetch()) allows for smoother training by fetching the next batch of data while the current batch is being processed, reducing idle time and improving GPU/CPU utilization.
</sub></CENTER>


In [10]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])


## Data Augmentation
a) Resizing the image to the standard size we defined earlier (256x256). Why are we doing it again, even though we did it before?
=> This is because the subsequent layer we create will be used in our final model. When our model makes predictions, if it encounters an image that isn't 256x256, it will handle it appropriately.

b) Dividing RGB values by 255 is a form of normalization that scales pixel values to a range between 0 and 1. This normalization aids in training machine learning models, particularly neural networks, by ensuring uniformity in input feature scales. It also helps prevent numerical instability and overflow/underflow issues during computations, promoting faster convergence and improved model performance.

c) RandomFlip and RandomRotation is important for image classification because it introduces variations in the training data, such as random flips and rotations. These variations help the model generalize better by learning from a diverse set of examples, making it more robust to different orientations and perspectives in real-world images.y.

In [11]:
input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
batch_input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
chanDim = -1
if K.image_data_format() == "channels_first":
    input_shape = (CHANNELS, IMAGE_SIZE, IMAGE_SIZE)
    batch_input_shape = (BATCH_SIZE, CHANNELS, IMAGE_SIZE, IMAGE_SIZE)
    chanDim = 1

In [12]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 3

model = models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)

## Implementing Data Augmentation to Train Dataset


In [13]:
model.summary()

In [14]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [15]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=EPOCHS,
)


👇
model.compile sets up the training configuration for the model, including how it learns (optimizer), how it measures performance (loss function), and what metrics to track during training. Each parameter is carefully chosen to optimize model training and improve classification accuracy.

In [16]:
print("[INFO] Calculating model accuracy")
scores = model.evaluate(test_ds)
print(f"Test Accuracy: {round(scores[1],4)*100}%")

In [ ]:
def apply_quantization(layer):
    if (
        isinstance(layer, layers.Dense)
        or isinstance(layer, layers.MaxPool2D)
        or isinstance(layer, layers.Conv2D)
    ):
        return tfmot.quantization.keras.quantize_annotate_layer(layer)
    return layer



In [ ]:
annotated_model = tf.keras.models.clone_model(
    model,
    clone_function=apply_quantization,
)

quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)
quant_aware_model.summary()

👇
using expand_dims to add an extra dimension at index 0 creates a batch of one image, which is necessary when working with models that expect input data in batches, even if you're processing a single image. This ensures compatibility between the input data shape and the model's input requirements.

Why It's Necessary:
Machine learning models, especially deep learning models, are often designed to process data in batches for efficiency and parallelization.
Even if you're working with a single image during inference (making predictions), the model expects input data in batch format.
Adding this extra dimension ensures that the input data conforms to the expected batch format, allowing the model to process the image correct

ly.

In [ ]:
quant_aware_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

## Running prediction on few images

In [ ]:
q_history = quant_aware_model.fit(train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=20,
)


## Saving the model

In [ ]:
print("[INFO] Calculating Quant Aware model accuracy")
scores = quant_aware_model.evaluate(test_ds)
print(f"Test Accuracy: {round(scores[1],4)*100}%")

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()
